<a href="https://colab.research.google.com/github/kkokay07/pq-genetics/blob/main/Local%20ancestry%20analysis/LAAX_(Local_Ancestry_Analysis_in_Crossbreds).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Local Ancestry Analysis in Crossbred (LAAX) Notebook - Tips to run

This notebook provides a comprehensive pipeline for analyzing local ancestry in crossbred populations. It implements the LAMP (Local Ancestry in adMixed Populations) method to identify genomic regions with significant ancestry deviations that may be associated with selection during breed formation.

## Prerequisites

Before running this notebook, you need:
1. A Google account with access to Google Drive
2. The following files uploaded to your Google Drive:
   - Google drive folder (e.g., `test`) containing binary PLINK files (.bed, .bim, .fam)
   - Google drive folder (e.g., `tools_local_ancestry`) containing LAMP executable and other tools

## Tips for Running the Notebook

- Run cells sequentially from top to bottom
- **After running Step 2, check that your parameters were correctly captured**
- The notebook will create multiple files throughout the analysis - review outputs at each stage
- The final results folder will contain visualizations and reports summarizing your findings


## Brief information on steps

The notebook is organized in a step-by-step workflow:

1. **Configuration Setup** (Step 1-2):
   - Mount your Google Drive
   - Set folder paths
   - Input parameters (ancestral populations, target population, file prefix)

2. **Quality Control and Filtering** (Step 3-4):
   - Filter SNPs based on MAF, HWE, and missingness
   - Perform admixture analysis to identify and remove outlier individuals

3. **Ancestry Analysis** (Step 5-7):
   - Calculate ancestral allele frequencies
   - Prepare LAMP input files
   - Run LAMP analysis for each chromosome

4. **Results and Visualization** (Step 8-10):
   - Calculate delta values (ancestry deviations)
   - Identify significant regions
   - Create visualizations and final reports


## Expected Outputs

The analysis will produce:
- Admixture visualizations showing population structure
- PCA plots showing genetic relationships
- Chromosome-specific delta files indicating ancestry deviation
- Heatmaps and summary statistics for significant regions
- A final HTML report summarizing all findings
- Produced folders (final_results, chromosome_plots and significant_regions) in the project folder (where your input files were present)

## Troubleshooting

- If a step fails, check the error message and ensure all required files exist
- Verify that executables have proper permissions (the notebook attempts to set these)
- For large datasets, some steps may take considerable time - be patient


In [ ]:
# Step 1: Mount Google Drive, set folder paths, and set permissions for all contents
from google.colab import drive
import os

# Mount your Google Drive (you'll be prompted to authorize)
drive.mount('/content/drive')

# Define your folder paths (adjust these paths based on your Drive structure)
PROJECT_FOLDER = '/content/drive/MyDrive/test'
TOOLS_FOLDER = '/content/drive/MyDrive/tools_local_ancestry'

# Check that the folders exist
if os.path.exists(PROJECT_FOLDER):
    print("Project Folder found:", PROJECT_FOLDER)
else:
    print("Project Folder not found. Please check your path.")

if os.path.exists(TOOLS_FOLDER):
    print("Tools Folder found:", TOOLS_FOLDER)
else:
    print("Tools Folder not found. Please check your path.")

# Set executable permissions for all files and directories in both folders
!chmod -R 755 "$PROJECT_FOLDER"
!chmod -R 755 "$TOOLS_FOLDER"

print("Permissions set for all contents in both folders.")

In [ ]:
# Step 2: Interactive Parameter Input
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create input widgets for parameters
anc1_widget = widgets.Text(value="Ggj", description="Ancestor 1:")
anc2_widget = widgets.Text(value="Ggd", description="Ancestor 2:")
target_widget = widgets.Text(value="Ggg", description="Target Name:")
prefix_widget = widgets.Text(value="Ggg=GgjxGgd", description="File Prefix:")
chr_set_widget = widgets.IntText(value=26, description="Chr Set:")

# Create a confirmation button
confirm_button = widgets.Button(description="Confirm Parameters", button_style='success')

# Global configuration dictionary to store the parameters
global_config = {}

def confirm_params(b):
    global global_config
    global_config = {
        "anc1": anc1_widget.value,
        "anc2": anc2_widget.value,
        "target": target_widget.value,
        "file_prefix": prefix_widget.value,
        "chr_set": chr_set_widget.value
    }
    clear_output()  # Clear the widget display after confirmation
    print("Configuration confirmed:")
    for key, value in global_config.items():
        print(f"{key}: {value}")

# Attach the callback to the button
confirm_button.on_click(confirm_params)

# Display the parameter input widgets
param_widgets = widgets.VBox([anc1_widget, anc2_widget, target_widget, prefix_widget, chr_set_widget, confirm_button])
display(param_widgets)

In [ ]:
# Step 3: Filtering Samples Using PLINK
import os
import subprocess
import pandas as pd

# Ensure PLINK executable exists
plink_path = os.path.join(TOOLS_FOLDER, "plink")
if not os.path.exists(plink_path):
    raise FileNotFoundError(f"❌ Error: PLINK not found at {plink_path}")

# Ensure global_config is loaded
try:
    if not global_config:
        raise ValueError("Global configuration not found. Please run Step 2 to confirm parameters.")
except NameError:
    raise ValueError("Global configuration not found. Please run Step 2 to confirm parameters.")

# Define file prefixes and paths
input_prefix = os.path.join(PROJECT_FOLDER, global_config["file_prefix"])  # Full path to input files
output_prefix = os.path.join(PROJECT_FOLDER, global_config["file_prefix"] + "_filtered")  # Output path

# Check if required input files exist
for ext in [".bed", ".bim", ".fam"]:
    file_path = f"{input_prefix}{ext}"
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"❌ Error: Missing required file: {file_path}")

# Define filtering thresholds
maf_threshold = 0.05    # Minor Allele Frequency threshold
hwe_threshold = 1e-6    # Hardy-Weinberg Equilibrium threshold
geno_threshold = 0.1    # SNP missingness threshold
mind_threshold = 0.1    # Individual missingness threshold
max_chr = global_config["chr_set"]  # Number of chromosomes

# Construct PLINK command
plink_cmd = f"""
{plink_path} --bfile {input_prefix} \
--chr-set {max_chr} \
--maf {maf_threshold} \
--hwe {hwe_threshold} \
--geno {geno_threshold} \
--mind {mind_threshold} \
--make-bed --out {output_prefix}
"""

print("Running PLINK filtering with the following command:")
print(plink_cmd)

# Execute PLINK command
result = subprocess.run(plink_cmd, shell=True, capture_output=True, text=True)

# Print PLINK output and error messages
print("PLINK Standard Output:")
print(result.stdout)

if result.stderr:
    print("PLINK Standard Error:")
    print(result.stderr)

# Verify that the filtered .fam file was created and count the remaining samples
filtered_fam_file = f"{output_prefix}.fam"
if os.path.exists(filtered_fam_file):
    fam_df = pd.read_csv(filtered_fam_file, sep=r"\s+", header=None)
    num_remaining_samples = fam_df.shape[0]
    print(f"✅ Number of samples remaining after filtering: {num_remaining_samples}")

    # Save the sample count for use in later steps
    remaining_samples_file = os.path.join(PROJECT_FOLDER, "remaining_samples.txt")
    with open(remaining_samples_file, "w") as f:
        f.write(str(num_remaining_samples))
    print(f"✅ Remaining samples count saved at: {remaining_samples_file}")
else:
    print(f"❌ Error: Filtered .fam file ({filtered_fam_file}) not found. Check PLINK output or log file.")

In [ ]:
# Step 4: Admixture and PLINK - Following the workflow in try20.ipynb
import os
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Helper function to run shell commands
def run_command(command):
    print(f"\nRunning command: {command}")
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    print("Command output:")
    print(result.stdout)
    if result.returncode != 0:
        print("Command error:")
        print(result.stderr)
    return result

# Use configuration from global_config
try:
    # global_config keys: file_prefix, target, anc1, anc2, chr_set
    input_prefix = global_config["file_prefix"]
    filtered_prefix = input_prefix + "_filtered"
    target_label = global_config["target"]
    anc1_label = global_config["anc1"]
    anc2_label = global_config["anc2"]
    max_chr = global_config["chr_set"]
except NameError:
    raise ValueError("global_config not found. Please run the interactive parameter cell (Step 2) first.")

# Define paths to executables and file paths
tools_dir = TOOLS_FOLDER
plink_path = os.path.join(TOOLS_FOLDER, "plink")
admixture_path = os.path.join(TOOLS_FOLDER, "admixture")

# Build paths with PROJECT_FOLDER
filtered_path = os.path.join(PROJECT_FOLDER, filtered_prefix)
print(f"Using filtered files: {filtered_path}")

# ------------------------------
# Step: Run ADMIXTURE at K=3
# ------------------------------
K3 = 3
admixture_cmd_K3 = f"{admixture_path} {filtered_path}.bed {K3}"
print(f"Running ADMIXTURE at K=3:")
run_command(admixture_cmd_K3)

# Look for Q file in multiple locations
q_file_K3 = f"{filtered_path}.{K3}.Q"
if not os.path.exists(q_file_K3):
    # Check if it's in the current directory
    local_q_file = f"{os.path.basename(filtered_prefix)}.{K3}.Q"
    if os.path.exists(local_q_file):
        q_file_K3 = local_q_file
        print(f"Found K=3 Q file in current directory: {q_file_K3}")
    else:
        raise FileNotFoundError(f"ADMIXTURE Q file {q_file_K3} not found!")

# Load the K=3 Q file
q3 = pd.read_csv(q_file_K3, delim_whitespace=True, header=None, dtype=float)
q3.columns = ["col0", "col1", "col2"]

# Load the .fam file
fam_cols = ["pop", "iid", "father", "mother", "sex", "phenotype"]
fam = pd.read_csv(f"{filtered_path}.fam", sep=r"\s+", header=None, names=fam_cols)

# Combine Q file with fam info (assuming row order matches)
admix_K3 = pd.concat([fam[["pop", "iid"]], q3], axis=1)

# ------------------------------
# Step: Assign Columns for K=3
# ------------------------------
# Identify "pure" ancestral individuals based on the pop labels
pure_BI = admix_K3[admix_K3["pop"] == anc1_label]
pure_BT = admix_K3[admix_K3["pop"] == anc2_label]

if pure_BI.empty or pure_BT.empty:
    print("Warning: One or both ancestral groups have no individuals. Using default column assignments.")
    BI_col = "col0"
    target_col = "col1"
    BT_col = "col2"
else:
    BI_avgs = pure_BI[["col0", "col1", "col2"]].mean()
    BT_avgs = pure_BT[["col0", "col1", "col2"]].mean()
    BI_col = BI_avgs.idxmax()  # assign column with highest average for anc1
    BT_col = BT_avgs.idxmax()  # assign column with highest average for anc2
    all_cols = {"col0", "col1", "col2"}
    target_col = list(all_cols - {BI_col, BT_col})[0]

print(f"Assigned columns (K=3):")
print(f"  {BI_col} -> {anc1_label}")
print(f"  {target_col} -> {target_label}")
print(f"  {BT_col} -> {anc2_label}")

# Plot K=3 admixture results
plt.figure(figsize=(12, 5))
sort_order = admix_K3.sort_values("pop").index
pop_colors = {'col0': 'red', 'col1': 'green', 'col2': 'blue'}

# Create stacked bar chart
bottom = np.zeros(len(admix_K3))
for col in ['col0', 'col1', 'col2']:
    values = admix_K3.loc[sort_order, col].values
    plt.bar(range(len(values)), values, bottom=bottom, width=0.8,
            color=pop_colors[col])
    bottom += values

# Add population labels
pop_groups = admix_K3.loc[sort_order].groupby("pop")
tick_positions = []
tick_labels = []
start_pos = 0
for pop, group in pop_groups:
    group_size = len(group)
    tick_positions.append(start_pos + group_size/2)
    tick_labels.append(f"{pop} (n={group_size})")
    start_pos += group_size
    # Draw vertical line between populations
    if start_pos < len(admix_K3):
        plt.axvline(x=start_pos - 0.5, color='black', linestyle='--', alpha=0.3)

plt.xticks(tick_positions, tick_labels, rotation=45, ha='right')
plt.ylabel("Ancestry Proportion")
plt.title("ADMIXTURE Results (K=3)")
plt.tight_layout()
plt.savefig(os.path.join(PROJECT_FOLDER, "admixture_K3_results.png"), dpi=300)
plt.show()

# Determine assignment for each individual (which Q column is highest)
admix_K3["assigned"] = admix_K3[[BI_col, target_col, BT_col]].idxmax(axis=1)

# ------------------------------
# Step: Filter Individuals Based on Composition Thresholds (K=3)
# ------------------------------
anc_threshold = 0.75    # For ancestral groups, require proportion >= 0.75
target_threshold = 0.90 # For target, remove if proportion > 0.90

# Filter ancestral BI group (assigned to BI)
group_BI = admix_K3[admix_K3["assigned"] == BI_col]
removed_BI = group_BI[group_BI[BI_col] < anc_threshold]
group_BI_filtered = group_BI[group_BI[BI_col] >= anc_threshold]

# Filter ancestral BT group (assigned to BT)
group_BT = admix_K3[admix_K3["assigned"] == BT_col]
removed_BT = group_BT[group_BT[BT_col] < anc_threshold]
group_BT_filtered = group_BT[group_BT[BT_col] >= anc_threshold]

# For the target population (e.g., PAH), remove individuals with high target proportion
group_target = admix_K3[admix_K3["pop"] == target_label]
removed_target = group_target[group_target[target_col] > target_threshold]
group_target_filtered = group_target[group_target[target_col] <= target_threshold]

print("\nFiltering results (K=3):")
print(f"{anc1_label} group: {len(group_BI)} before, {len(removed_BI)} removed (<{anc_threshold}), {len(group_BI_filtered)} remaining.")
print(f"{anc2_label} group: {len(group_BT)} before, {len(removed_BT)} removed (<{anc_threshold}), {len(group_BT_filtered)} remaining.")
print(f"{target_label} group: {len(group_target)} before, {len(removed_target)} removed (>{target_threshold}), {len(group_target_filtered)} remaining.")

# Combine IDs of individuals to remove (from any group failing the thresholds)
remove_ids = pd.concat([removed_BI[["pop", "iid"]],
                       removed_BT[["pop", "iid"]],
                       removed_target[["pop", "iid"]]])
remove_ids_file = os.path.join(PROJECT_FOLDER, "remove_ids.txt")
remove_ids.to_csv(remove_ids_file, sep="\t", header=False, index=False)
print(f"\nRemoval list saved to {remove_ids_file} (total individuals to remove: {remove_ids.shape[0]})")

# ------------------------------
# Step: Create a New Filtered Dataset Using PLINK --remove
# ------------------------------
new_filtered_prefix = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_UM_filtered")
plink_remove_cmd = f"{plink_path} --cow --bfile {filtered_path} --remove {remove_ids_file} --make-bed --out {new_filtered_prefix}"
print("\nRunning PLINK --remove command:")
run_command(plink_remove_cmd)

new_fam_file = f"{new_filtered_prefix}.fam"
if os.path.exists(new_fam_file):
    new_fam_df = pd.read_csv(new_fam_file, sep=r"\s+", header=None)
    new_sample_count = new_fam_df.shape[0]
    print(f"New sample count after removal: {new_sample_count}")
else:
    print("New filtered .fam file not found. Check PLINK removal command.")

# ------------------------------
# Step: Re-run ADMIXTURE at K=2 on the New Filtered Dataset
# ------------------------------
K2 = 2
admixture_cmd_K2 = f"{admixture_path} {new_filtered_prefix}.bed {K2}"
print("\nRunning ADMIXTURE at K=2 with command:")
print(admixture_cmd_K2)
run_command(admixture_cmd_K2)

q_file_K2 = f"{new_filtered_prefix}.{K2}.Q"
if not os.path.exists(q_file_K2):
    # Check if it's in the current directory with a different path
    local_q_file = f"{os.path.basename(new_filtered_prefix)}.{K2}.Q"
    if os.path.exists(local_q_file):
        q_file_K2 = local_q_file
        print(f"Found K=2 Q file in current directory: {q_file_K2}")
    else:
        raise FileNotFoundError(f"ADMIXTURE Q file {q_file_K2} not found after removal!")

# Load the K=2 Q file
q2 = pd.read_csv(q_file_K2, delim_whitespace=True, header=None, dtype=float)
q2.columns = ["col0", "col1"]

# Load the new .fam file with population info
new_fam = pd.read_csv(new_fam_file, delim_whitespace=True, header=None, names=fam_cols)

# Combine Q file with fam info for K=2
q2_full = pd.concat([new_fam[["pop", "iid"]], q2], axis=1)

# Determine column assignments for K=2 using pure ancestral individuals
pure_BI_new = q2_full[q2_full["pop"] == anc1_label]
if not pure_BI_new.empty:
    BI_avgs_new = pure_BI_new[["col0", "col1"]].mean()
    BI_col_K2 = BI_avgs_new.idxmax()  # assign column with highest average among BI individuals as BI
else:
    print("Warning: No individuals for ancestral group", anc1_label, "in new filtered file. Defaulting to col0.")
    BI_col_K2 = "col0"
all2 = {"col0", "col1"}
BT_col_K2 = list(all2 - {BI_col_K2})[0]

# Rename Q file columns to ancestral labels
q2_full = q2_full.rename(columns={BI_col_K2: anc1_label, BT_col_K2: anc2_label})

# Compute average ancestry for target individuals in K=2
target_new = q2_full[q2_full["pop"] == target_label]
target_anc1_avg_new = target_new[anc1_label].mean() if not target_new.empty else np.nan
target_anc2_avg_new = target_new[anc2_label].mean() if not target_new.empty else np.nan

print("\nADMIXTURE (K=2) average ancestry in target population:")
print(f"  {anc1_label}: {target_anc1_avg_new:.4f}")
print(f"  {anc2_label}: {target_anc2_avg_new:.4f}")

# ------------------------------
# Updated ADMIXTURE (K=2) Bar Plot with Grouped x-axis
# ------------------------------
# Sort samples by population to group them
q2_full_sorted = q2_full.sort_values("pop").reset_index(drop=True)

# Determine x positions with gaps between populations
pop_groups = q2_full_sorted.groupby("pop")
x_positions = []         # x position for each sample
tick_positions = {}      # x position for labeling each population group
current_pos = 0
gap = 1  # gap between groups (in x coordinate units)
for pop, group in pop_groups:
    group_length = group.shape[0]
    positions = list(range(current_pos, current_pos + group_length))
    x_positions.extend(positions)
    tick_positions[pop] = current_pos + (group_length - 1) / 2
    current_pos += group_length + gap

x_positions = np.array(x_positions)
bottom_vals = np.zeros(len(q2_full_sorted))
colors = ["red", "blue"]

plt.figure(figsize=(12, 5), dpi=300)
for i, col in enumerate([anc1_label, anc2_label]):
    plt.bar(x_positions, q2_full_sorted[col], bottom=bottom_vals, color=colors[i], width=0.8)
    bottom_vals += q2_full_sorted[col].values

plt.ylabel("Ancestry Proportion")
plt.xticks(list(tick_positions.values()), list(tick_positions.keys()), rotation=45)
plt.xlim(min(x_positions)-0.5, max(x_positions)+0.5)
plt.title("ADMIXTURE Results (K=2)")
plt.tight_layout()
admixture_plot_K2 = os.path.join(PROJECT_FOLDER, "admixture_barplot_filtered_K2.png")
plt.savefig(admixture_plot_K2, dpi=300)
plt.show()
print(f"ADMIXTURE K=2 plot saved as {admixture_plot_K2}")

# ------------------------------
# Run PCA on the New Filtered Dataset
# ------------------------------
pca_out_path = os.path.join(PROJECT_FOLDER, "pca_out_filtered_K2")
pca_cmd_K2 = f"{plink_path} --cow --nonfounders --bfile {new_filtered_prefix} --pca 2 --out {pca_out_path}"
print("\nRunning PCA on new filtered dataset (K=2):")
run_command(pca_cmd_K2)

pca_file_K2 = f"{pca_out_path}.eigenvec"
if os.path.exists(pca_file_K2):
    pca_df_K2 = pd.read_csv(pca_file_K2, delim_whitespace=True, header=None)
    pca_df_K2.columns = ["FID", "IID", "PC1", "PC2"]
    pca_merged_K2 = pd.merge(pca_df_K2, new_fam[["pop", "iid"]], left_on="IID", right_on="iid")
    plt.figure(figsize=(8,6), dpi=300)
    unique_pops = pca_merged_K2["pop"].unique()
    pop_colors = {pop: col for pop, col in zip(unique_pops, ["red", "blue", "green", "orange", "purple"])}
    for pop in unique_pops:
        subset = pca_merged_K2[pca_merged_K2["pop"] == pop]
        plt.scatter(subset["PC1"], subset["PC2"], label=pop, color=pop_colors.get(pop, "grey"), s=50, alpha=0.7)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.legend(title="Population", loc="best")
    plt.title("PCA of Filtered Dataset (K=2)")
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    pca_plot_K2 = os.path.join(PROJECT_FOLDER, "pca_plot_filtered_K2.png")
    plt.savefig(pca_plot_K2, dpi=300)
    plt.show()
    print(f"PCA plot (K=2) saved as {pca_plot_K2}")
else:
    print("PCA eigenvec file (K=2) not found.")

# ------------------------------
# Step: Save Summary Information to a LaTeX File
# ------------------------------
original_target_count = len(fam[fam["pop"] == target_label])
filtered_target_count = len(target_new) if 'target_new' in locals() and not target_new.empty else np.nan

latex_file = os.path.join(PROJECT_FOLDER, "ancestry_summary.tex")
with open(latex_file, "w") as f:
    f.write(r"\documentclass{article}" + "\n")
    f.write(r"\begin{document}" + "\n\n")
    f.write(r"\section*{ADMIXTURE and PCA Summary}" + "\n")
    f.write("Number of target samples before filtering: " + str(original_target_count) + r"\\")
    f.write("\nNumber of target samples after filtering (K=2): " + str(filtered_target_count) + r"\\")
    f.write("\n\\vspace{0.5cm}" + "\n")
    f.write(r"\begin{tabular}{lcc}" + "\n")
    f.write(r"\hline" + "\n")
    f.write("Population & Avg. Ancestry (K=3) & Avg. Ancestry (K=2) \\\\" + "\n")
    f.write(r"\hline" + "\n")
    if not group_target_filtered.empty:
        target_anc1_avg_K3 = group_target_filtered[BI_col].mean()
        target_anc2_avg_K3 = group_target_filtered[target_col].mean()
    else:
        target_anc1_avg_K3, target_anc2_avg_K3 = np.nan, np.nan
    f.write(f"{anc1_label} & {target_anc1_avg_K3:.4f} & {target_anc1_avg_new:.4f} \\\\" + "\n")
    f.write(f"{anc2_label} & {target_anc2_avg_K3:.4f} & {target_anc2_avg_new:.4f} \\\\" + "\n")
    f.write(r"\hline" + "\n")
    f.write(r"\end{tabular}" + "\n\n")
    f.write(r"\end{document}" + "\n")

print(f"\nLaTeX summary file '{latex_file}' created.")

In [ ]:
# Step 5: Calculate Ancestral Allele Frequencies
import os
import subprocess
import pandas as pd


# Use configuration from global_config
try:
    # Retrieve configuration from global_config (set in Step 2)
    input_prefix    = global_config["file_prefix"]
    filtered_prefix = input_prefix + "_filtered"
    target_label    = global_config["target"]
    anc_pop1_label  = global_config["anc1"]
    anc_pop2_label  = global_config["anc2"]
    max_chr         = global_config["chr_set"]
except NameError:
    raise ValueError("global_config not found. Please run the interactive parameter cell (Step 2) first.")

# Define the path to PLINK executable
tools_dir = TOOLS_FOLDER
plink_path = os.path.join(TOOLS_FOLDER, "plink")

# Build proper paths for input and output files
filtered_path = os.path.join(PROJECT_FOLDER, filtered_prefix)
output_freq_prefix = os.path.join(PROJECT_FOLDER, "Afreq_LocaBreed")

############################################
# Step: Calculate Ancestral Allele Frequencies using PLINK
############################################
freq_cmd = f'{plink_path} --cow --nonfounders --bfile {filtered_path} --freq --family --out {output_freq_prefix}'
print("Running PLINK for allele frequency calculation:")
subprocess.run(freq_cmd, shell=True, check=True)
print("Allele frequency calculation completed.")

############################################
# Step: Process Allele Frequencies File
############################################
freq_file = f"{output_freq_prefix}.frq.strat"
if not os.path.exists(freq_file):
    raise FileNotFoundError(f"Frequency file {freq_file} not found!")

afreq = pd.read_csv(freq_file, delim_whitespace=True)
# Expected columns in .frq.strat (e.g.): CHR, SNP, A1, A2, MAF, NCHROBS, CLST
# Compute allele frequency for allele A: by default use MAF
afreq["AlleleA"] = afreq["MAF"]
# If the alternate allele (A2) is coded as "2", then flip the frequency
afreq.loc[afreq["A2"] == "2", "AlleleA"] = 1 - afreq.loc[afreq["A2"] == "2", "MAF"]
print("Processed allele frequencies.")

# Display population distribution in frequency file
print("\nPopulation distribution in frequency file:")
pop_counts = afreq["CLST"].value_counts()
for pop, count in pop_counts.items():
    print(f"  {pop}: {count} records")

############################################
# Step: Write Ancestral Allele Frequency Files per Chromosome
############################################
# Subset frequency file by ancestral population using the CLST column
afreq_anc1 = afreq[afreq["CLST"] == anc_pop1_label]
afreq_anc2 = afreq[afreq["CLST"] == anc_pop2_label]

print(f"\nWriting allele frequency files per chromosome:")
for i in range(1, max_chr + 1):
    out_file_anc1 = os.path.join(PROJECT_FOLDER, f"purebred_{anc_pop1_label}_CHR{i}.prob")
    out_file_anc2 = os.path.join(PROJECT_FOLDER, f"purebred_{anc_pop2_label}_CHR{i}.prob")

    # Write allele frequencies (AlleleA) for SNPs on chromosome i
    data_anc1 = afreq_anc1.loc[afreq_anc1["CHR"] == i, "AlleleA"]
    data_anc2 = afreq_anc2.loc[afreq_anc2["CHR"] == i, "AlleleA"]

    data_anc1.to_csv(out_file_anc1, sep="\t", header=False, index=False, quoting=3)
    data_anc2.to_csv(out_file_anc2, sep="\t", header=False, index=False, quoting=3)
    print(f"  Chromosome {i}:")
    print(f"    {os.path.basename(out_file_anc1)} ({len(data_anc1)} SNPs)")
    print(f"    {os.path.basename(out_file_anc2)} ({len(data_anc2)} SNPs)")

############################################
# Step: Create Keep File for Target Individuals
############################################
# Read the filtered .fam file
fam_file = f"{filtered_path}.fam"
fam_cols = ["pop", "iid", "father", "mother", "sex", "phenotype"]
fam_df = pd.read_csv(fam_file, sep=r"\s+", header=None, names=fam_cols)

# Select individuals whose population equals target_label
target_keep = fam_df[fam_df["pop"] == target_label][["pop", "iid"]]
target_keep_file = os.path.join(PROJECT_FOLDER, f"{target_label}_crossbreds.keep")
target_keep.to_csv(target_keep_file, sep="\t", header=False, index=False)
print(f"\nCreated keep file '{os.path.basename(target_keep_file)}' with {target_keep.shape[0]} individuals.")

############################################
# Calculate ancestral sample sizes from the filtered .fam file
############################################
pure_anc1_count = fam_df[fam_df["pop"] == anc_pop1_label].shape[0]
pure_anc2_count = fam_df[fam_df["pop"] == anc_pop2_label].shape[0]
print(f"Ancestral sample sizes: {anc_pop1_label} = {pure_anc1_count}, {anc_pop2_label} = {pure_anc2_count}")

############################################
# Step: Extract Each Chromosome from the Filtered Dataset
############################################
print("\nExtracting chromosome-specific datasets:")
for i in range(1, max_chr + 1):
    chr_out = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{i}")
    cmd = f'{plink_path} --cow --nonfounders --bfile {filtered_path} --chr {i} --make-bed --out {chr_out}'
    print(f"  Chromosome {i}... ", end="", flush=True)
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print("success")
    else:
        print("failed")
        print(f"  Error: {result.stderr}")

############################################
# Step: Create Recode Allele Input Files for Each Chromosome
############################################
print("\nCreating recode files for each chromosome:")
for i in range(1, max_chr + 1):
    bim_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{i}.bim")
    recode_file = os.path.join(PROJECT_FOLDER, f"recodeAB_CHR{i}.txt")

    if os.path.exists(bim_file):
        bim_df = pd.read_csv(bim_file, delim_whitespace=True, header=None)
        # In the PLINK .bim file, column 2 (index 1) is the SNP ID.
        # Create a DataFrame with SNP ID and a constant "2"
        recode_df = pd.DataFrame({
            0: bim_df.iloc[:, 1],
            1: ["2"] * bim_df.shape[0]
        })
        recode_df.to_csv(recode_file, sep="\t", header=False, index=False, quoting=3)
        print(f"  Created recode file for chromosome {i} with {bim_df.shape[0]} SNPs")
    else:
        print(f"  Skipping chromosome {i} - BIM file not found")

############################################
# Step: Recode Genotypes for Each Chromosome Using PLINK
############################################
print("\nRecoding genotypes for each chromosome:")
for i in range(1, max_chr + 1):
    chr_in = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{i}")
    recode_file = os.path.join(PROJECT_FOLDER, f"recodeAB_CHR{i}.txt")

    if os.path.exists(f"{chr_in}.bed") and os.path.exists(recode_file):
        cmd = f'{plink_path} --cow --nonfounders --bfile {chr_in} --keep {target_keep_file} --reference-allele {recode_file} --recode A --out {chr_in}'
        print(f"  Chromosome {i}... ", end="", flush=True)
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            print("success")
            # Check if the output file was created
            if os.path.exists(f"{chr_in}.raw"):
                print(f"    Created {os.path.basename(chr_in)}.raw")
            else:
                print(f"    Warning: {os.path.basename(chr_in)}.raw not found")
        else:
            print("failed")
            print(f"    Error: {result.stderr}")
    else:
        print(f"  Skipping chromosome {i} - required input files not found")

print("\nCompleted all steps for calculating ancestral allele frequencies.")
print(f"Files are saved in: {PROJECT_FOLDER}")

# Print summary of the files created
print("\nSummary of key files created:")
print(f"1. Allele frequency file: {os.path.basename(output_freq_prefix)}.frq.strat")
print(f"2. Ancestral allele probability files: purebred_{anc_pop1_label}_CHR*.prob, purebred_{anc_pop2_label}_CHR*.prob")
print(f"3. Target individuals keep file: {os.path.basename(target_keep_file)}")
print(f"4. Chromosome-specific datasets: {os.path.basename(filtered_prefix)}_CHR*.bed/.bim/.fam")
print(f"5. Recode files: recodeAB_CHR*.txt")
print(f"6. Recoded genotypes: {os.path.basename(filtered_prefix)}_CHR*.raw")

In [ ]:
# Step 6: Prepare LAMP Input Files
import os
import re
import pandas as pd
from IPython.display import clear_output

# Use configuration from global_config
try:
    # Retrieve configuration from global_config
    input_prefix    = global_config["file_prefix"]
    filtered_prefix = input_prefix + "_filtered"
    target_label    = global_config["target"]
    anc_pop1_label  = global_config["anc1"]
    anc_pop2_label  = global_config["anc2"]
    max_chr         = global_config["chr_set"]
except NameError:
    raise ValueError("global_config not found. Please run the interactive parameter cell (Step 2) first.")

# Determine the final filtered prefix after admixture-based filtering
final_filtered_prefix = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_UM_filtered")

# Extract alpha values from ancestry_summary.tex
def extract_alpha_from_tex(tex_file, population_labels):
    try:
        with open(tex_file, 'r') as f:
            content = f.read()

        pattern = re.compile(rf"{population_labels[0]}\\s+&\\s+[\\d.]+\\s+&\\s+([\\d.]+).*?{population_labels[1]}\\s+&\\s+[\\d.]+\\s+&\\s+([\\d.]+)", re.DOTALL)
        match = pattern.search(content)

        if match:
            alpha1, alpha2 = float(match.group(1)), float(match.group(2))
            total = alpha1 + alpha2
            alpha1 /= total
            alpha2 /= total
            return alpha1, alpha2
        else:
            # If we can't find the exact pattern, look for any floating point numbers after the labels
            alt_pattern1 = re.compile(rf"{population_labels[0]}.*?([0-9.]+)")
            alt_pattern2 = re.compile(rf"{population_labels[1]}.*?([0-9.]+)")

            match1 = alt_pattern1.search(content)
            match2 = alt_pattern2.search(content)

            if match1 and match2:
                alpha1, alpha2 = float(match1.group(1)), float(match2.group(1))
                total = alpha1 + alpha2
                alpha1 /= total
                alpha2 /= total
                return alpha1, alpha2
            else:
                print(f"Warning: Could not extract ancestry proportions from {tex_file}")
                # Return default values
                return 0.5, 0.5
    except Exception as e:
        print(f"Error reading {tex_file}: {str(e)}")
        return 0.5, 0.5

# Fetch ancestral sample sizes from the filtered .fam file
def fetch_ancestral_sizes(fam_file, anc_labels):
    try:
        fam_df = pd.read_csv(fam_file, delim_whitespace=True, header=None)
        anc_size1 = fam_df[fam_df[0] == anc_labels[0]].shape[0]
        anc_size2 = fam_df[fam_df[0] == anc_labels[1]].shape[0]
        return anc_size1, anc_size2
    except Exception as e:
        print(f"Error fetching ancestral sizes from {fam_file}: {str(e)}")
        return 10, 10  # Default values if file can't be read

# Function to prepare LAMP input files
def prepare_lamp_inputs(chrom, base_prefix, alpha1, alpha2, anc_size1, anc_size2, anc_labels):
    prefix = f"{base_prefix}_filtered"
    raw_file = os.path.join(PROJECT_FOLDER, f"{prefix}_CHR{chrom}.raw")
    bim_file = os.path.join(PROJECT_FOLDER, f"{prefix}_CHR{chrom}.bim")
    lampgeno_file = os.path.join(PROJECT_FOLDER, f"{prefix}_CHR{chrom}_LAMPGENO.txt")
    lampmap_file = os.path.join(PROJECT_FOLDER, f"{prefix}_CHR{chrom}_LAMPMAP.txt")
    config_file = os.path.join(PROJECT_FOLDER, f"{prefix}_CHR{chrom}_configfile.txt")

    # LAMPGENO file - extract genotype data from raw file
    if os.path.exists(raw_file):
        try:
            with open(raw_file, 'r') as infile:
                lines = infile.readlines()

            if len(lines) > 1:
                with open(lampgeno_file, 'w') as outfile:
                    for line in lines[1:]:  # Skip header
                        parts = line.strip().split()
                        geno = parts[6:]  # Skip FID,IID,PAT,MAT,SEX,PHENOTYPE columns
                        # Replace missing values with -1
                        geno = ["-1" if x.upper() == "NA" else x for x in geno]
                        outfile.write(" ".join(geno) + "\n")
                print(f"[CHR{chrom}] Created LAMPGENO: {os.path.basename(lampgeno_file)}")
            else:
                print(f"[CHR{chrom}] {raw_file} has only a header row or is empty. No data to process.")
                return False
        except Exception as e:
            print(f"[CHR{chrom}] Error processing raw file: {str(e)}")
            return False
    else:
        print(f"[CHR{chrom}] Raw file not found: {raw_file}")
        return False

    # LAMPMAP file - extract position data from bim file
    if os.path.exists(bim_file):
        try:
            with open(bim_file, 'r') as infile, open(lampmap_file, 'w') as outfile:
                for line in infile:
                    parts = line.strip().split()
                    if len(parts) >= 4:
                        # Column 4 (index 3) in BIM file contains the position
                        outfile.write(parts[3] + "\n")
            print(f"[CHR{chrom}] Created LAMPMAP: {os.path.basename(lampmap_file)}")
        except Exception as e:
            print(f"[CHR{chrom}] Error processing BIM file: {str(e)}")
            return False
    else:
        print(f"[CHR{chrom}] BIM file not found: {bim_file}")
        return False

    # Create LAMP configuration file
    prob_file1 = os.path.join(PROJECT_FOLDER, f"purebred_{anc_labels[0]}_CHR{chrom}.prob")
    prob_file2 = os.path.join(PROJECT_FOLDER, f"purebred_{anc_labels[1]}_CHR{chrom}.prob")

    config_content = (
        "populations=2\n"
        f"pfile={prob_file1},{prob_file2}\n"
        f"ancestralsamplesize={anc_size1},{anc_size2}\n"
        f"genofile={lampgeno_file}\n"
        f"posfile={lampmap_file}\n"
        f"outputancestryfile={os.path.join(PROJECT_FOLDER, f'{prefix}_CHR{chrom}_results.txt')}\n"
        "offset=0.2\n"
        "recombrate=1e-8\n"
        "generations=02\n"
        f"alpha={alpha1:.4f},{alpha2:.4f}\n"
        "ldcutoff=1\n"
    )

    with open(config_file, 'w') as f:
        f.write(config_content)
    print(f"[CHR{chrom}] Created LAMP config: {os.path.basename(config_file)}")
    return True

# Main execution
print("Step 6: Preparing LAMP Input Files")
print("="*50)

# Fetch alpha values from ancestry_summary.tex
tex_file_path = os.path.join(PROJECT_FOLDER, "ancestry_summary.tex")
alpha1, alpha2 = extract_alpha_from_tex(tex_file_path, [anc_pop1_label, anc_pop2_label])
print(f"Ancestry proportions extracted: {anc_pop1_label}={alpha1:.4f}, {anc_pop2_label}={alpha2:.4f}")

# Fetch ancestral sample sizes
fam_file_path = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_UM_filtered.fam")
if not os.path.exists(fam_file_path):
    fam_file_path = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}.fam")
    print(f"Warning: Using original filtered .fam file: {os.path.basename(fam_file_path)}")

anc_size1, anc_size2 = fetch_ancestral_sizes(fam_file_path, [anc_pop1_label, anc_pop2_label])
print(f"Ancestral sample sizes: {anc_pop1_label}={anc_size1}, {anc_pop2_label}={anc_size2}")

# Check if required files exist
print("\nChecking for required input files:")
files_missing = False
for i in range(1, max_chr + 1):
    raw_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{i}.raw")
    bim_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{i}.bim")
    prob_file1 = os.path.join(PROJECT_FOLDER, f"purebred_{anc_pop1_label}_CHR{i}.prob")
    prob_file2 = os.path.join(PROJECT_FOLDER, f"purebred_{anc_pop2_label}_CHR{i}.prob")

    chr_status = []
    for f, desc in [
        (raw_file, "raw file"),
        (bim_file, "bim file"),
        (prob_file1, f"{anc_pop1_label} prob file"),
        (prob_file2, f"{anc_pop2_label} prob file")
    ]:
        if os.path.exists(f):
            chr_status.append(f"✓ {desc}")
        else:
            chr_status.append(f"✗ {desc}")
            files_missing = True

    print(f"  Chromosome {i}: " + ", ".join(chr_status))

if files_missing:
    print("\nWarning: Some required files are missing. LAMP preparation may be incomplete.")
else:
    print("\nAll required files for LAMP analysis are present.")

# Loop over chromosomes and prepare LAMP input files
print("\nPreparing LAMP input files for each chromosome:")
prepared_count = 0
for i in range(1, max_chr + 1):
    print(f"\n=== Preparing LAMP input files for Chromosome {i} ===")
    success = prepare_lamp_inputs(i, input_prefix, alpha1, alpha2, anc_size1, anc_size2, [anc_pop1_label, anc_pop2_label])
    if success:
        prepared_count += 1

print(f"\nSuccessfully prepared LAMP input files for {prepared_count} out of {max_chr} chromosomes")

# Print summary of configuration for LAMP
print("\nLAMP Configuration Summary:")
print(f"  Populations: 2 ({anc_pop1_label}, {anc_pop2_label})")
print(f"  Ancestry proportions: {anc_pop1_label}={alpha1:.4f}, {anc_pop2_label}={alpha2:.4f}")
print(f"  Ancestral sample sizes: {anc_pop1_label}={anc_size1}, {anc_pop2_label}={anc_size2}")
print("  Other parameters:")
print("    - offset: 0.2")
print("    - recombination rate: 1e-8")
print("    - generations: 2")
print("    - LD cutoff: 1")

# Create a shell script to run LAMP for all chromosomes
lamp_script_file = os.path.join(PROJECT_FOLDER, "run_lamp.sh")
with open(lamp_script_file, 'w') as f:
    f.write("#!/bin/bash\n\n")
    f.write(f"LAMP_PATH=\"{os.path.join(TOOLS_FOLDER, 'lamp')}\"\n\n")
    f.write("# Run LAMP for each chromosome\n")
    for i in range(1, max_chr + 1):
        config_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{i}_configfile.txt")
        f.write(f"echo \"Processing chromosome {i}...\"\n")
        f.write(f"\"$LAMP_PATH\" {config_file}\n")
        f.write("echo \"Done with chromosome ${i}\"\n\n")

# Make the script executable
os.chmod(lamp_script_file, 0o755)
print(f"\nCreated a shell script to run LAMP: {os.path.basename(lamp_script_file)}")
print("You can run this script to execute LAMP on all chromosomes")

In [ ]:
# Step 7: Run LAMP Analysis for Each Chromosome
import os
import subprocess
import time

# Use configuration from global_config
try:
    # Retrieve configuration from global_config
    input_prefix    = global_config["file_prefix"]
    filtered_prefix = input_prefix + "_filtered"
    target_label    = global_config["target"]
    anc_pop1_label  = global_config["anc1"]
    anc_pop2_label  = global_config["anc2"]
    max_chr         = global_config["chr_set"]
except NameError:
    raise ValueError("global_config not found. Please run the interactive parameter cell (Step 2) first.")

# Define the LAMP executable path
lamp_exe = os.path.join(TOOLS_FOLDER, "lamp")

# Check if LAMP executable exists
if not os.path.exists(lamp_exe):
    print(f"Warning: LAMP executable not found at {lamp_exe}")
    lamp_exe = "lamp"  # Try using the command directly
    print(f"Will try using 'lamp' command directly. Ensure it's in your PATH.")

# Function to run LAMP for a single chromosome
def run_lamp_for_chromosome(chrom):
    config_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{chrom}_configfile.txt")
    results_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{chrom}_results.txt")

    # Check if config file exists
    if not os.path.exists(config_file):
        print(f"Error: Config file not found for chromosome {chrom}: {config_file}")
        return False

    # Check if results file already exists
    if os.path.exists(results_file):
        print(f"Results file already exists for chromosome {chrom}. Skipping.")
        return True

    # Run LAMP
    print(f"Running LAMP for Chromosome {chrom}...")
    start_time = time.time()

    lamp_cmd = f"{lamp_exe} {config_file}"
    result = subprocess.run(lamp_cmd, shell=True, capture_output=True, text=True)

    # Check if the command was successful
    if result.returncode != 0:
        print(f"Error running LAMP for chromosome {chrom}:")
        print(result.stderr)
        return False

    # Check if results file was created
    if os.path.exists(results_file):
        end_time = time.time()
        duration = end_time - start_time
        file_size = os.path.getsize(results_file) / 1024  # Size in KB
        print(f"LAMP completed for chromosome {chrom} in {duration:.1f} seconds.")
        print(f"Results file size: {file_size:.1f} KB")
        return True
    else:
        print(f"LAMP ran but results file not found for chromosome {chrom}!")
        return False

# Print header
print("=" * 60)
print("Step 7: Running LAMP Analysis for Each Chromosome")
print("=" * 60)

# Check required files before starting
print("\nChecking required files before starting LAMP analysis:")
files_ready = True

for chrom in range(1, max_chr + 1):
    config_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{chrom}_configfile.txt")
    lampgeno_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{chrom}_LAMPGENO.txt")
    lampmap_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{chrom}_LAMPMAP.txt")
    prob_file1 = os.path.join(PROJECT_FOLDER, f"purebred_{anc_pop1_label}_CHR{chrom}.prob")
    prob_file2 = os.path.join(PROJECT_FOLDER, f"purebred_{anc_pop2_label}_CHR{chrom}.prob")

    missing_files = []
    for file_path, file_type in [
        (config_file, "config"),
        (lampgeno_file, "LAMPGENO"),
        (lampmap_file, "LAMPMAP"),
        (prob_file1, f"{anc_pop1_label} prob"),
        (prob_file2, f"{anc_pop2_label} prob")
    ]:
        if not os.path.exists(file_path):
            missing_files.append(file_type)

    if missing_files:
        print(f"  Chromosome {chrom}: Missing {', '.join(missing_files)} files")
        files_ready = False
    else:
        print(f"  Chromosome {chrom}: All required files present ✓")

if not files_ready:
    print("\nWarning: Some required files are missing. Make sure to complete Step 6 (Prepare LAMP Input Files) first.")
    print("Proceeding anyway, but some chromosomes might be skipped.")

# Run LAMP for each chromosome
print("\nRunning LAMP analysis for each chromosome:")
successful_chrs = []
failed_chrs = []

for chrom in range(1, max_chr + 1):
    print(f"\nProcessing chromosome {chrom}...")
    success = run_lamp_for_chromosome(chrom)
    if success:
        successful_chrs.append(chrom)
    else:
        failed_chrs.append(chrom)

# Print summary
print("\n" + "=" * 60)
print("LAMP Analysis Summary")
print("=" * 60)
print(f"Total chromosomes processed: {max_chr}")
print(f"Successfully completed: {len(successful_chrs)} chromosomes")
if successful_chrs:
    print(f"  Chromosomes: {', '.join(map(str, successful_chrs))}")
print(f"Failed: {len(failed_chrs)} chromosomes")
if failed_chrs:
    print(f"  Chromosomes: {', '.join(map(str, failed_chrs))}")

print("\nNext steps:")
if successful_chrs:
    print("- Proceed to Step 8 (Calculate Delta) to process the LAMP results")
    print("- The delta values will be used to identify regions with ancestry enrichment")
else:
    print("- Revisit the previous steps to ensure all input files are correctly prepared")
    print("- Check that the LAMP executable is working properly")

# Create a file with the list of completed chromosomes for the next step
completed_file = os.path.join(PROJECT_FOLDER, "completed_chromosomes.txt")
with open(completed_file, 'w') as f:
    f.write('\n'.join(map(str, successful_chrs)))
print(f"\nSaved list of successfully completed chromosomes to {os.path.basename(completed_file)}")

In [ ]:
# Step 8: Calculate Delta from LAMP Results (Fixed)
import os
import pandas as pd
import numpy as np

# Use configuration from global_config
try:
    # Retrieve configuration from global_config
    input_prefix    = global_config["file_prefix"]
    filtered_prefix = input_prefix + "_filtered"
    target_label    = global_config["target"]
    anc_pop1_label  = global_config["anc1"]
    anc_pop2_label  = global_config["anc2"]
    max_chr         = global_config["chr_set"]
except NameError:
    raise ValueError("global_config not found. Please run the interactive parameter cell (Step 2) first.")

# Print header
print("=" * 60)
print("Step 8: Calculate Delta from LAMP Results (Fixed)")
print("=" * 60)

# Determine which chromosomes have completed LAMP analysis
completed_chrs = []
for i in range(1, max_chr + 1):
    results_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{i}_results.txt")
    if os.path.exists(results_file):
        completed_chrs.append(i)

if not completed_chrs:
    raise FileNotFoundError("No LAMP results files found. Please run Step 7 (LAMP Analysis) first.")

print(f"Found LAMP results for {len(completed_chrs)} chromosomes: {', '.join(map(str, completed_chrs))}")

# Step 1: Determine the number of target individuals
fam_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}.fam")
fam_cols = ["pop", "iid", "father", "mother", "sex", "phenotype"]
if not os.path.exists(fam_file):
    raise FileNotFoundError(f"FAM file not found: {fam_file}")

fam_df = pd.read_csv(fam_file, sep=r"\s+", header=None, names=fam_cols)
target_df = fam_df[fam_df["pop"] == target_label]
n_target = target_df.shape[0]
print(f"Found {n_target} target individuals in {os.path.basename(fam_file)} (population: {target_label}).")

# Looking at the LAMP results file format:
# - Each individual has 2 rows (one for each haplotype)
# - The first column has "ID:" followed by tab-separated values
# - We need to parse this manually

# Function to parse LAMP results file
def parse_lamp_results(results_file, odd_indices_only=True):
    """
    Parse LAMP results file, which has the format:
    ID: val1 val2 val3 ...

    Parameters:
    - results_file: Path to the LAMP results file
    - odd_indices_only: If True, only return odd-indexed rows (first haplotype of each individual)

    Returns:
    DataFrame with ancestry values
    """
    try:
        with open(results_file, 'r') as f:
            lines = f.readlines()

        # Extract data rows
        data_rows = []
        for line in lines:
            if line.strip():  # Skip empty lines
                parts = line.strip().split('\t')
                if len(parts) > 1:
                    # Extract ID and values
                    id_part = parts[0].strip()
                    id_num = int(id_part.split(':')[0]) if ':' in id_part else -1
                    values = [float(v) for v in parts[1:]]
                    data_rows.append((id_num, values))

        # Convert to DataFrame
        if not data_rows:
            return None

        # If we only want odd-indexed rows (for the first haplotype of each individual)
        if odd_indices_only:
            # Find the pattern of indices
            unique_ids = sorted(set(row[0] for row in data_rows))
            if len(unique_ids) > 1:
                # Determine if we have a pattern of 2 rows per individual
                # If so, take only the first row of each pair
                rows_per_individual = 2
                selected_rows = data_rows[::rows_per_individual]
            else:
                # If we can't detect a pattern, just return all rows
                selected_rows = data_rows
        else:
            selected_rows = data_rows

        # Extract just the values
        values_only = [row[1] for row in selected_rows]
        return pd.DataFrame(values_only)

    except Exception as e:
        print(f"Error parsing LAMP results file: {str(e)}")
        return None

# Step 2: Calculate global mean ancestry across all chromosomes
print("\nCalculating global mean ancestry across all chromosomes...")
all_snp_means = []

# First pass: Collect all SNP means from all chromosomes
for chr_num in completed_chrs:
    print(f"  Processing chromosome {chr_num}... ", end="", flush=True)

    # Build file names using filtered_prefix
    results_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{chr_num}_results.txt")

    # Parse the LAMP results file
    res_df = parse_lamp_results(results_file)
    if res_df is None or res_df.empty:
        print("Error: Could not parse results file or file is empty")
        continue

    # Calculate mean ancestry across individuals
    snp_means = res_df.mean(axis=0)
    all_snp_means.extend(snp_means.tolist())
    print(f"added {len(snp_means)} SNP means")

# Calculate the global mean from all SNP means across all chromosomes
if all_snp_means:
    global_mean = np.mean(all_snp_means)
    global_sd = np.std(all_snp_means)
    print(f"\nGlobal mean ancestry across all chromosomes: {global_mean:.4f}")
    print(f"Global standard deviation: {global_sd:.4f}")
else:
    print("No SNP means were collected. Cannot calculate global mean.")
    global_mean = 0
    global_sd = 0

# Step 3: Process Each Chromosome to Compute Delta using the global mean
merged_list = []

for chr_num in completed_chrs:
    print(f"\n=== Processing Chromosome {chr_num} ===")

    # Build file names using filtered_prefix
    results_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{chr_num}_results.txt")
    lampmap_file = os.path.join(PROJECT_FOLDER, f"{filtered_prefix}_CHR{chr_num}_LAMPMAP.txt")

    if not os.path.exists(results_file) or not os.path.exists(lampmap_file):
        print(f"Required files not found for chromosome {chr_num}. Skipping.")
        continue

    # Parse the LAMP results file
    res_df = parse_lamp_results(results_file)
    if res_df is None or res_df.empty:
        print(f"Error: Could not parse results file for chromosome {chr_num} or file is empty")
        continue

    # Calculate mean ancestry across individuals
    snp_means = res_df.mean(axis=0)

    # Calculate delta using the global mean across all chromosomes
    delta = snp_means - global_mean

    # Read positions from LAMPMAP file
    try:
        lampmap = pd.read_csv(lampmap_file, sep="\t", header=None)
        if lampmap.shape[1] != 1:
            print(f"Warning: Unexpected format in LAMPMAP file for chromosome {chr_num}")

        # Convert positions from basepairs to megabases (Mb)
        lampmap[0] = lampmap[0] / 1e6
    except Exception as e:
        print(f"Error reading LAMPMAP file for chromosome {chr_num}: {str(e)}")
        continue

    # Check if the number of SNPs matches; if not, truncate to the minimum length
    len_delta = len(delta)
    len_map = len(lampmap)
    if len_delta != len_map:
        print(f"Warning: Mismatch in SNP counts for chromosome {chr_num}: {len_delta} vs. {len_map}. Truncating to minimum length.")
        min_length = min(len_delta, len_map)
        delta = delta.iloc[:min_length]
        lampmap = lampmap.iloc[:min_length]

    # Create a DataFrame with delta values and SNP positions
    chr_delta_df = pd.DataFrame({
        "admix_rate": delta.values,
        "position": lampmap[0].values,
        "chromosome": chr_num
    })

    # Save per-chromosome delta file
    out_filename = os.path.join(PROJECT_FOLDER, f"{target_label}_{anc_pop1_label}_{anc_pop2_label}_CHR{chr_num}.txt")
    chr_delta_df.to_csv(out_filename, sep="\t", index=False)
    print(f"Saved delta file for chromosome {chr_num} as {os.path.basename(out_filename)}")

    merged_list.append(chr_delta_df)

# Step 4: Merge All Chromosome Delta Files into One
if merged_list:
    merged_df = pd.concat(merged_list, ignore_index=True)
    merged_output = os.path.join(PROJECT_FOLDER, f"{target_label}_{anc_pop1_label}_{anc_pop2_label}_merged.txt")
    merged_df.to_csv(merged_output, sep="\t", index=False)
    print(f"\nMerged delta file created: {os.path.basename(merged_output)}")
    print(f"Total SNPs in merged file: {merged_df.shape[0]}")

    # Display the first few rows of the merged file to confirm format
    print("\nFirst few rows of the merged file:")
    print(merged_df.head().to_string())

    # Calculate summary statistics for the deltas
    mean_delta = merged_df['admix_rate'].mean()
    sd_delta = merged_df['admix_rate'].std()
    min_delta = merged_df['admix_rate'].min()
    max_delta = merged_df['admix_rate'].max()

    print("\nDelta Statistics:")
    print(f"  Mean: {mean_delta:.4f}")
    print(f"  Standard Deviation: {sd_delta:.4f}")
    print(f"  Min: {min_delta:.4f}")
    print(f"  Max: {max_delta:.4f}")

    # Calculate enrichment thresholds
    threshold_2sd = 2 * sd_delta
    threshold_3sd = 3 * sd_delta

    print(f"\nEnrichment Thresholds:")
    print(f"  2 SD (+/-): {threshold_2sd:.4f}")
    print(f"  3 SD (+/-): {threshold_3sd:.4f}")

    # Count regions exceeding thresholds
    exceed_2sd = merged_df[abs(merged_df['admix_rate']) > threshold_2sd]
    exceed_3sd = merged_df[abs(merged_df['admix_rate']) > threshold_3sd]

    print(f"\nRegions exceeding thresholds:")
    print(f"  > 2 SD: {exceed_2sd.shape[0]} SNPs ({exceed_2sd.shape[0]/merged_df.shape[0]*100:.2f}%)")
    print(f"  > 3 SD: {exceed_3sd.shape[0]} SNPs ({exceed_3sd.shape[0]/merged_df.shape[0]*100:.2f}%)")

    # Save threshold values for the next step
    thresholds_file = os.path.join(PROJECT_FOLDER, "delta_thresholds.txt")
    with open(thresholds_file, 'w') as f:
        f.write(f"mean\t{mean_delta}\n")
        f.write(f"sd\t{sd_delta}\n")
        f.write(f"threshold_2sd\t{threshold_2sd}\n")
        f.write(f"threshold_3sd\t{threshold_3sd}\n")

    print(f"\nThreshold values saved to {os.path.basename(thresholds_file)}")
else:
    print("No delta data available to merge.")

In [ ]:
# Step 9: Visualization and Identification of Significant Regions
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Use configuration from global_config
try:
    # Retrieve configuration from global_config
    input_prefix    = global_config["file_prefix"]
    filtered_prefix = input_prefix + "_filtered"
    target_label    = global_config["target"]
    anc_pop1_label  = global_config["anc1"]
    anc_pop2_label  = global_config["anc2"]
    max_chr         = global_config["chr_set"]
except NameError:
    raise ValueError("global_config not found. Please run the interactive parameter cell (Step 2) first.")

# Print header
print("=" * 60)
print("Step 9: Visualization and Identification of Significant Regions")
print("=" * 60)

# Build the merged file name according to previous step output
merged_filename = os.path.join(PROJECT_FOLDER, f"{target_label}_{anc_pop1_label}_{anc_pop2_label}_merged.txt")
if not os.path.exists(merged_filename):
    raise FileNotFoundError(f"Merged delta file not found: {merged_filename}")

# Load the merged delta data
print(f"Loading merged delta data from {os.path.basename(merged_filename)}")
data = pd.read_csv(merged_filename, sep="\t")
print(f"Loaded {data.shape[0]} SNPs across {data['chromosome'].nunique()} chromosomes")

# Define a function to process and save data for significant regions
def identify_significant_regions(chromosome_data):
    # Calculate the mean and standard deviation of admix_rate
    mean_admix = chromosome_data['admix_rate'].mean()
    sd_admix = chromosome_data['admix_rate'].std()

    # Set the threshold to 3 standard deviations
    threshold = 3 * sd_admix

    # Identify rows exceeding the threshold
    exceed_threshold = chromosome_data[abs(chromosome_data['admix_rate']) > threshold]

    return exceed_threshold, threshold

# Create directory for output files
output_dir = os.path.join(PROJECT_FOLDER, "significant_regions")
os.makedirs(output_dir, exist_ok=True)
print(f"Saving significant region results to {output_dir}")

# Process each chromosome and identify significant regions
print("\nIdentifying significant regions by chromosome:")
all_significant_regions = []

for chromosome in sorted(data['chromosome'].unique()):
    chromosome_data = data[data['chromosome'] == chromosome]
    exceed_threshold, threshold = identify_significant_regions(chromosome_data)

    if not exceed_threshold.empty:
        # Save to file
        file_name = os.path.join(output_dir, f"chromosome_{chromosome}_exceed_threshold.txt")
        exceed_threshold.to_csv(file_name, sep="\t", index=False, quoting=3)

        # Add to combined results
        all_significant_regions.append(exceed_threshold)

        print(f"  Chromosome {chromosome}: {len(exceed_threshold)} significant SNPs (threshold = {threshold:.4f})")
    else:
        print(f"  Chromosome {chromosome}: No significant SNPs found")

# Combine all significant regions into one file
if all_significant_regions:
    combined = pd.concat(all_significant_regions, ignore_index=True)
    combined_file = os.path.join(output_dir, "all_significant_regions.txt")
    combined.to_csv(combined_file, sep="\t", index=False, quoting=3)
    print(f"\nCombined all significant regions into {os.path.basename(combined_file)}")
    print(f"Total significant SNPs across all chromosomes: {len(combined)}")
else:
    print("\nNo significant regions found in any chromosome")

# Create directory for plots
plots_dir = os.path.join(PROJECT_FOLDER, "chromosome_plots")
os.makedirs(plots_dir, exist_ok=True)
print(f"\nPreparing multi-chromosome grid plots...")

# Define a function to plot chromosome in grid layout
def plot_chromosome_in_grid(ax, chromosome_data, chromosome):
    # Calculate the mean and standard deviation of admix_rate
    mean_admix = chromosome_data['admix_rate'].mean()
    sd_admix = chromosome_data['admix_rate'].std()
    threshold = 3 * sd_admix

    # Create the line plot
    ax.plot(chromosome_data['position'], chromosome_data['admix_rate'], color='blue')

    # Highlight regions exceeding the threshold
    exceed_threshold = chromosome_data[abs(chromosome_data['admix_rate']) > threshold]
    if not exceed_threshold.empty:
        ax.scatter(exceed_threshold['position'], exceed_threshold['admix_rate'], color='red')

    # Add zero line (dotted green)
    ax.axhline(y=0, color='green', linestyle=':', alpha=0.8)

    # Add threshold lines (dashed orange) without values
    ax.axhline(y=threshold, color='orange', linestyle='--')
    ax.axhline(y=-threshold, color='orange', linestyle='--')

    # Set x-axis labels and avoid overlapping
    min_pos = chromosome_data['position'].min()
    max_pos = chromosome_data['position'].max()
    # Use first, last and either middle point or space them evenly
    if max_pos - min_pos > 50:
        positions = [min_pos, min_pos + (max_pos - min_pos)/2, max_pos]
    else:
        positions = [min_pos, max_pos]

    positions = [round(pos, 2) for pos in positions]

    # Format labels to prevent overlapping
    if max(positions) > 100:
        x_labels = [f"{p:.0f}" for p in positions]  # No decimal for large values
    else:
        x_labels = [f"{p:.2f}" for p in positions]

    ax.set_xticks(positions)
    ax.set_xticklabels(x_labels, fontsize=8)

    # Set title
    ax.set_title(f"Chromosome {chromosome}")

    # Set axis labels
    ax.set_xlabel("Position (cM)", fontsize=8)
    if ax.get_subplotspec().is_first_col():  # Only for leftmost column
        ax.set_ylabel("Delta Ancestry", fontsize=8)

    # Set y-axis limits to accommodate thresholds plus a little padding
    max_y = max(threshold * 1.2, chromosome_data['admix_rate'].max() * 1.2)
    min_y = min(-threshold * 1.2, chromosome_data['admix_rate'].min() * 1.2)
    ax.set_ylim(min_y, max_y)

    # Add a faint border
    for spine in ax.spines.values():
        spine.set_linewidth(0.5)
        spine.set_color('#888888')

# Create multi-chromosome grid plots
print("\nCreating multi-chromosome grid plots...")
chromosomes = sorted(data['chromosome'].unique())
num_chromosomes = len(chromosomes)
chromosomes_per_page = 16  # 16 per page (4x4 grid)

# Calculate total number of pages needed
total_pages = (num_chromosomes + chromosomes_per_page - 1) // chromosomes_per_page

# Define consistent figure size and layout for all pages
figsize = (15, 12)

# Process all pages with consistent formatting
for page in range(total_pages):
    page_num = page + 1
    start_idx = page * chromosomes_per_page
    end_idx = min(start_idx + chromosomes_per_page, num_chromosomes)

    # Create a figure with consistent dimensions for all pages
    fig, axes = plt.subplots(4, 4, figsize=figsize)
    axes = axes.flatten()

    # Process chromosomes for this page
    for i, idx in enumerate(range(start_idx, end_idx)):
        chromosome = chromosomes[idx]
        chromosome_data = data[data['chromosome'] == chromosome]
        plot_chromosome_in_grid(axes[i], chromosome_data, chromosome)

    # Set empty plots for any remaining slots
    for j in range(end_idx - start_idx, 16):
        axes[j].axis('off')

    # Add title with consistent formatting
    plt.suptitle(f"Ancestry Deviation by Chromosome - Page {page_num}", fontsize=14, y=0.98)

    # Add legend only to the first plot of each page
    if page_num == 1:  # For first page only
        # Add a legend to the top right of the figure
        legend_elements = [
            plt.Line2D([0], [0], color='orange', linestyle='--', label='3SD Threshold'),
            plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=8, label='Exceed 3SD Threshold')
        ]
        fig.legend(handles=legend_elements, loc='upper right', ncol=2, frameon=False,
                  bbox_to_anchor=(0.98, 0.98), fontsize=10)

    # Ensure consistent spacing and layout
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Save the grid plot with consistent naming
    output_file = os.path.join(plots_dir, f"chromosomes_grid_page_{page_num}.png")
    plt.savefig(output_file, dpi=300)
    print(f"  Created grid plot page {page_num}: {os.path.basename(output_file)}")
    plt.close()

print("\nVisualization complete. All plots saved successfully.")

# Create a summary file with counts of significant regions by chromosome
summary_data = []
for chromosome in sorted(data['chromosome'].unique()):
    chromosome_data = data[data['chromosome'] == chromosome]
    exceed_threshold, threshold = identify_significant_regions(chromosome_data)

    summary_data.append({
        'Chromosome': chromosome,
        'Total_SNPs': len(chromosome_data),
        'Significant_SNPs': len(exceed_threshold),
        'Threshold': threshold,
        'Percent_Significant': (len(exceed_threshold) / len(chromosome_data) * 100) if len(chromosome_data) > 0 else 0
    })

summary_df = pd.DataFrame(summary_data)
summary_file = os.path.join(PROJECT_FOLDER, "significance_summary.txt")
summary_df.to_csv(summary_file, sep="\t", index=False)
print(f"\nSaved significance summary to {os.path.basename(summary_file)}")

In [ ]:
# Step 10: Final Analysis and Reporting
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Use configuration from global_config
try:
    # Retrieve configuration from global_config
    input_prefix    = global_config["file_prefix"]
    filtered_prefix = input_prefix + "_filtered"
    target_label    = global_config["target"]
    anc_pop1_label  = global_config["anc1"]
    anc_pop2_label  = global_config["anc2"]
    max_chr         = global_config["chr_set"]
except NameError:
    raise ValueError("global_config not found. Please run the interactive parameter cell (Step 2) first.")

# Print header
print("=" * 60)
print("Step 10: Final Analysis and Reporting")
print("=" * 60)

# Define important file paths
merged_file = os.path.join(PROJECT_FOLDER, f"{target_label}_{anc_pop1_label}_{anc_pop2_label}_merged.txt")
significant_file = os.path.join(PROJECT_FOLDER, "significant_regions", "all_significant_regions.txt")
summary_file = os.path.join(PROJECT_FOLDER, "significance_summary.txt")

# Check if required files exist
required_files = [merged_file, significant_file, summary_file]
missing_files = [f for f in required_files if not os.path.exists(f)]

if missing_files:
    print(f"Warning: The following required files are missing: {', '.join(missing_files)}")
    print("Some analyses may be incomplete. Please run previous steps first.")
else:
    print("All required input files found.")

# Create directory for final outputs
final_dir = os.path.join(PROJECT_FOLDER, "final_results")
os.makedirs(final_dir, exist_ok=True)
print(f"Final results will be saved to: {final_dir}")

# Load the merged data file
try:
    print(f"\nLoading merged data from {os.path.basename(merged_file)}...")
    merged_data = pd.read_csv(merged_file, sep="\t")
    print(f"Loaded {merged_data.shape[0]} SNPs across {merged_data['chromosome'].nunique()} chromosomes")
except Exception as e:
    print(f"Error loading merged data: {str(e)}")
    merged_data = None

# Load the significant regions file if it exists
try:
    print(f"\nLoading significant regions from {os.path.basename(significant_file)}...")
    sig_regions = pd.read_csv(significant_file, sep="\t")
    print(f"Loaded {sig_regions.shape[0]} significant SNPs")
except Exception as e:
    print(f"Error or no significant regions file: {str(e)}")
    sig_regions = None

# Load the summary file
try:
    print(f"\nLoading significance summary from {os.path.basename(summary_file)}...")
    summary = pd.read_csv(summary_file, sep="\t")
    print(f"Loaded summary statistics for {summary.shape[0]} chromosomes")
except Exception as e:
    print(f"Error loading summary file: {str(e)}")
    summary = None

# PART 1: Create chromosome-wide heatmap visualization
if merged_data is not None:
    print("\n1. Creating chromosome-wide heatmap visualization...")

    # Prepare data for heatmap
    chromosomes = sorted(merged_data['chromosome'].unique())

    # Create a figure with a heatmap-style representation
    plt.figure(figsize=(15, 8))

    # For each chromosome, calculate the percentage of significant regions in bins
    bin_size = 1  # in Mb (adjust as needed)
    max_chr_length = 0
    all_binned_data = []

    for chromosome in chromosomes:
        chr_data = merged_data[merged_data['chromosome'] == chromosome]
        if chr_data.empty:
            continue

        # Find chromosome length
        chr_length = int(np.ceil(chr_data['position'].max()))
        max_chr_length = max(max_chr_length, chr_length)

        # Create bins
        bins = np.arange(0, chr_length + bin_size, bin_size)
        binned_data = np.zeros(len(bins) - 1)

        # Calculate mean delta for each bin
        for i in range(len(bins) - 1):
            bin_start, bin_end = bins[i], bins[i+1]
            bin_snps = chr_data[(chr_data['position'] >= bin_start) & (chr_data['position'] < bin_end)]

            if not bin_snps.empty:
                binned_data[i] = bin_snps['admix_rate'].mean()

        # Pad with zeros to make all chromosomes the same length
        all_binned_data.append(np.pad(binned_data, (0, int(np.ceil(max_chr_length/bin_size)) - len(binned_data)), 'constant'))

    # Create a heatmap
    heatmap_data = np.array(all_binned_data)

    # Create a custom colormap with white at zero
    max_abs_val = np.max(np.abs(heatmap_data))
    colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue -> White -> Red
    cmap = LinearSegmentedColormap.from_list("BlueWhiteRed", colors, N=256)

    # Plot heatmap
    plt.imshow(heatmap_data, aspect='auto', cmap=cmap, vmin=-max_abs_val, vmax=max_abs_val)
    plt.colorbar(label=f"Mean Delta Ancestry ({anc_pop1_label} vs {anc_pop2_label})")

    # Set y-axis ticks to chromosome numbers
    plt.yticks(range(len(chromosomes)), [f"Chr {c}" for c in chromosomes])

    # Set x-axis to position in Mb
    tick_positions = np.arange(0, int(np.ceil(max_chr_length/bin_size)), 10)  # Every 10 Mb
    plt.xticks(tick_positions, [f"{p*bin_size}" for p in tick_positions])
    plt.xlabel("Position (Mb)")

    plt.title(f"Genome-wide Ancestry Deviations in {target_label}")
    plt.tight_layout()

    # Save the figure
    heatmap_file = os.path.join(final_dir, "genome_wide_heatmap.png")
    plt.savefig(heatmap_file, dpi=300)
    plt.close()
    print(f"Saved genome-wide heatmap to {os.path.basename(heatmap_file)}")

# PART 2: Analyze significant regions and create summary statistics
if sig_regions is not None:
    print("\n2. Analyzing significant regions...")

    # Count significant regions per chromosome and direction of deviation
    chromosome_stats = []

    for chromosome in sorted(sig_regions['chromosome'].unique()):
        chr_sig = sig_regions[sig_regions['chromosome'] == chromosome]

        # Count positive and negative deviations
        positive_count = sum(chr_sig['admix_rate'] > 0)
        negative_count = sum(chr_sig['admix_rate'] < 0)

        # Calculate the percentage of positive vs negative
        total = positive_count + negative_count
        pos_percent = (positive_count / total * 100) if total > 0 else 0
        neg_percent = (negative_count / total * 100) if total > 0 else 0

        # Get statistics on the magnitude of deviations
        mean_positive = chr_sig[chr_sig['admix_rate'] > 0]['admix_rate'].mean() if positive_count > 0 else 0
        mean_negative = chr_sig[chr_sig['admix_rate'] < 0]['admix_rate'].mean() if negative_count > 0 else 0
        max_positive = chr_sig['admix_rate'].max() if positive_count > 0 else 0
        min_negative = chr_sig['admix_rate'].min() if negative_count > 0 else 0

        chromosome_stats.append({
            'Chromosome': chromosome,
            f'{anc_pop1_label}_enriched': positive_count,
            f'{anc_pop2_label}_enriched': negative_count,
            'Total_significant': total,
            f'{anc_pop1_label}_percent': pos_percent,
            f'{anc_pop2_label}_percent': neg_percent,
            f'Mean_{anc_pop1_label}_enrichment': mean_positive,
            f'Mean_{anc_pop2_label}_enrichment': mean_negative,
            f'Max_{anc_pop1_label}_enrichment': max_positive,
            f'Min_{anc_pop2_label}_enrichment': min_negative
        })

    # Create a DataFrame and save
    chr_stats_df = pd.DataFrame(chromosome_stats)
    chr_stats_file = os.path.join(final_dir, "chromosome_statistics.txt")
    chr_stats_df.to_csv(chr_stats_file, sep="\t", index=False)
    print(f"Saved chromosome statistics to {os.path.basename(chr_stats_file)}")

    # Identify clusters of significant regions (potential selective sweeps)
    print("\n3. Identifying clusters of significant regions...")

    # Group adjacent significant SNPs into clusters
    clusters = []
    current_cluster = []

    # Sort significant regions by chromosome and position
    sorted_sig = sig_regions.sort_values(['chromosome', 'position'])

    # Set distance threshold for considering SNPs part of the same cluster (e.g., 1 Mb)
    distance_threshold = 1.0  # in Mb

    # Iterate through SNPs
    for i, row in sorted_sig.iterrows():
        if not current_cluster:
            # Start a new cluster with the first SNP
            current_cluster = [row]
        else:
            prev_row = current_cluster[-1]

            # Check if this SNP belongs to the current cluster
            if (row['chromosome'] == prev_row['chromosome'] and
                row['position'] - prev_row['position'] <= distance_threshold):
                current_cluster.append(row)
            else:
                # Save the completed cluster and start a new one
                if len(current_cluster) >= 3:  # Only keep clusters with at least 3 SNPs
                    clusters.append(pd.DataFrame(current_cluster))
                current_cluster = [row]

    # Don't forget the last cluster
    if current_cluster and len(current_cluster) >= 3:
        clusters.append(pd.DataFrame(current_cluster))

    # Save information about clusters
    if clusters:
        print(f"Found {len(clusters)} clusters of significant regions")

        # Save each cluster to a separate file
        os.makedirs(os.path.join(final_dir, "clusters"), exist_ok=True)

        # Create a summary of clusters
        cluster_summary = []

        for i, cluster in enumerate(clusters):
            # Calculate cluster statistics
            start_pos = cluster['position'].min()
            end_pos = cluster['position'].max()
            length = end_pos - start_pos
            chromosome = cluster['chromosome'].iloc[0]
            avg_delta = cluster['admix_rate'].mean()
            direction = f"{anc_pop1_label} enriched" if avg_delta > 0 else f"{anc_pop2_label} enriched"

            # Save cluster details
            cluster_file = os.path.join(final_dir, "clusters", f"cluster_{i+1}_chr{chromosome}.txt")
            cluster.to_csv(cluster_file, sep="\t", index=False)

            cluster_summary.append({
                'Cluster_ID': i+1,
                'Chromosome': chromosome,
                'Start_Position': start_pos,
                'End_Position': end_pos,
                'Length_Mb': length,
                'SNP_Count': len(cluster),
                'Mean_Delta': avg_delta,
                'Direction': direction
            })

        # Create a summary DataFrame and save
        cluster_summary_df = pd.DataFrame(cluster_summary)
        cluster_summary_file = os.path.join(final_dir, "cluster_summary.txt")
        cluster_summary_df.to_csv(cluster_summary_file, sep="\t", index=False)
        print(f"Saved cluster summary to {os.path.basename(cluster_summary_file)}")

        # Create a visualization of the top clusters
        top_clusters = cluster_summary_df.sort_values('SNP_Count', ascending=False).head(10)

        plt.figure(figsize=(12, 8))
        colors = ['red' if direction.startswith(anc_pop1_label) else 'blue' for direction in top_clusters['Direction']]

        plt.barh(range(len(top_clusters)), top_clusters['SNP_Count'], color=colors)

        # Add chromosome and position labels
        labels = [f"Chr{row['Chromosome']}: {row['Start_Position']:.1f}-{row['End_Position']:.1f} Mb"
                for _, row in top_clusters.iterrows()]

        plt.yticks(range(len(top_clusters)), labels)
        plt.xlabel('Number of Significant SNPs in Cluster')
        plt.title('Top 10 Clusters of Significant Ancestry Deviation')

        legend_elements = [
            plt.Rectangle((0,0), 1, 1, color='red', label=f'{anc_pop1_label} enriched'),
            plt.Rectangle((0,0), 1, 1, color='blue', label=f'{anc_pop2_label} enriched')
        ]
        plt.legend(handles=legend_elements)

        plt.tight_layout()

        # Save the figure
        top_clusters_file = os.path.join(final_dir, "top_clusters.png")
        plt.savefig(top_clusters_file, dpi=300)
        plt.close()
        print(f"Saved top clusters visualization to {os.path.basename(top_clusters_file)}")
    else:
        print("No significant clusters found")

# PART 3: Create a final report
print("\n4. Generating final report...")

# Create a final report in HTML format
report_file = os.path.join(final_dir, "final_report.html")

with open(report_file, 'w') as f:
    f.write(f'''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Local Ancestry Analysis Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; margin: 20px; }}
            h1, h2, h3 {{ color: #2c3e50; }}
            .container {{ max-width: 1200px; margin: 0 auto; }}
            table {{ border-collapse: collapse; width: 100%; margin-bottom: 20px; }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
            th {{ background-color: #f2f2f2; }}
            tr:nth-child(even) {{ background-color: #f9f9f9; }}
            .highlight {{ background-color: #ffffcc; }}
            img {{ max-width: 100%; height: auto; margin: 10px 0; }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Local Ancestry Analysis Report</h1>
            <p><strong>Date:</strong> {pd.Timestamp.now().strftime('%Y-%m-%d')}</p>

            <h2>Analysis Parameters</h2>
            <ul>
                <li><strong>Target Population:</strong> {target_label}</li>
                <li><strong>Ancestral Population 1:</strong> {anc_pop1_label}</li>
                <li><strong>Ancestral Population 2:</strong> {anc_pop2_label}</li>
                <li><strong>Number of Chromosomes:</strong> {max_chr}</li>
            </ul>

            <h2>Summary Statistics</h2>
    ''')

    # Include summary statistics if available
    if summary is not None:
        total_snps = summary['Total_SNPs'].sum()
        total_sig = summary['Significant_SNPs'].sum()
        pct_sig = (total_sig / total_snps * 100) if total_snps > 0 else 0

        f.write(f'''
            <p>Total SNPs analyzed: {total_snps:,}</p>
            <p>Significant SNPs: {total_sig:,} ({pct_sig:.2f}%)</p>

            <h3>Significant SNPs by Chromosome</h3>
            <table>
                <tr>
                    <th>Chromosome</th>
                    <th>Total SNPs</th>
                    <th>Significant SNPs</th>
                    <th>Percent</th>
                    <th>Threshold</th>
                </tr>
        ''')

        for _, row in summary.iterrows():
            f.write(f'''
                <tr>
                    <td>{row['Chromosome']}</td>
                    <td>{row['Total_SNPs']:,}</td>
                    <td>{row['Significant_SNPs']:,}</td>
                    <td>{row['Percent_Significant']:.2f}%</td>
                    <td>±{row['Threshold']:.4f}</td>
                </tr>
            ''')

        f.write('</table>')

    # Include distribution of enrichment if sig_regions is available
    if sig_regions is not None:
        positive_count = sum(sig_regions['admix_rate'] > 0)
        negative_count = sum(sig_regions['admix_rate'] < 0)
        total_sig = positive_count + negative_count

        f.write(f'''
            <h3>Distribution of Ancestry Enrichment</h3>
            <p>{anc_pop1_label} enriched regions: {positive_count:,} ({positive_count/total_sig*100:.2f}%)</p>
            <p>{anc_pop2_label} enriched regions: {negative_count:,} ({negative_count/total_sig*100:.2f}%)</p>
        ''')

    # Include images
    f.write('''
            <h2>Visualizations</h2>

            <h3>Genome-wide Ancestry Deviations</h3>
            <img src="genome_wide_heatmap.png" alt="Genome-wide heatmap">

            <h3>Top Clusters of Significant Deviation</h3>
            <img src="top_clusters.png" alt="Top clusters">
    ''')

    # Add conclusions
    f.write('''
            <h2>Conclusions</h2>
            <p>This analysis has identified regions of the genome where local ancestry shows significant deviation from the genome-wide average. These regions may represent:</p>
            <ul>
                <li>Areas under selection during the formation of the target population</li>
                <li>Regions with adaptive introgression from one ancestral population</li>
                <li>Genomic areas with ancestry-related incompatibilities</li>
            </ul>
            <p>Further investigation of the specific genes in these regions is recommended.</p>
        </div>
    </body>
    </html>
    ''')

print(f"Saved final report to {os.path.basename(report_file)}")
print("\nFinal analysis complete. Results are available in the following directory:")
print(f"{final_dir}")